In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')
!unzip "/content/drive/MyDrive/PCAM data/ new_train.zip"
#unzip files


In [ ]:
!unzip "/content/drive/MyDrive/PCAM data/ new_histotest.zip"

In [ ]:
!unzip "/content/drive/MyDrive/PCAM data/ histotest1.zip"

In [ ]:
import os
print(len(os.listdir("/content/new_train")))


In [ ]:
from fastai.vision import *

In [ ]:
from fastai.metrics import error_rate

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
train_dir="/content/new_train"
base_dir="/content"
train_csv="/content/drive/My Drive/train_labels1.csv"

In [ ]:
import numpy as np
import random 
import os
from tqdm import tqdm   
l=os.listdir(train_dir)
random.shuffle(l)

In [ ]:
tfms = get_transforms(do_flip=True) #augmentation

In [ ]:
len(os.listdir("/content/new_histotest"))

In [ ]:
bs=32 # also the default batch size
data = ImageDataBunch.from_csv(
    base_dir, 
    ds_tfms=tfms, 
    size=224, 
    suffix=".tiff",
    folder="new_histotest", 
    csv_labels="histotest.csv", 
    bs=bs)

In [ ]:
len(os.listdir(histotest1))

In [ ]:
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=4, figsize=(10, 10))

In [ ]:
learn = cnn_learner(data, models.densenet161, metrics=error_rate, callback_fns=ShowGraph).to_fp16() #to load model

#change parameter to models.{what model you want name} to load the model

In [ ]:
learn = learn.load("/content/drive/My Drive/densenet10epochs")
learn.data  = data

In [ ]:
torch.cuda.empty_cache #free gpu space



In [ ]:

learn.lr_find() # mocks training to plt graph of learning rate vs loss
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, slice(1e-05)) #parameters are number of epochs and learning rate choosen from above graph

In [ ]:
learn.save("densenet161final") #save model

In [ ]:
        learn=learn.load("/content/drive/My Drive/200k") #if you want to load previously saved models

In [ ]:
preds,y, loss = learn.get_preds(ds_type=DatasetType.Valid,with_loss=True) #validation accuracy
# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc*100))


In [ ]:
preds,y, loss = learn.get_preds(ds_type=DatasetType.Train,with_loss=True) #training accuracy
# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc*100))

In [ ]:
import h5py
f2 = h5py.File("/content/drive/My Drive/camelyonpatch_level_2_split_test_y.h5", 'r')

In [ ]:
set2 = f2['y']

In [ ]:
import cv2

In [ ]:
c=0
tp=0
tn=0
fp=0
fn=0
for i in range(32768):
    image=cv2.imread("/content/new_histotest/"+str(i)+".tiff")
    actual=open_image("/content/new_histotest/"+str(i)+".tiff")
    preds=learn.predict(actual)
    result=int(preds[0])
    print("result is",result,"actual is",set2[i][0][0][0])
    print(i)
    #plt.imshow(image)
    #plt.show()
    if(result==set2[i][0][0][0] and result==1 and set2[i][0][0][0]==1):
      tp=tp+1
      c=c+1
    elif(result==set2[i][0][0][0] and result==0 and set2[i][0][0][0]==0):
      tn=tn+1
      c=c+1
    elif(result!=set2[i][0][0][0] and result==1 and set2[i][0][0][0]==0):
      fp=fp+1
    else:
      fn=fn+1

In [ ]:
c

In [ ]:
c/32768

In [ ]:
c*100

In [ ]:
(c*100)/32768

In [ ]:
fp


In [ ]:
fn

In [ ]:
tp

In [ ]:
tn

In [ ]:
precision=tp/(tp+fp)

In [ ]:
recall=tp/(tp+fn)

In [ ]:
print(precision,recall)

In [ ]:
F1_Score=(2*precision*recall)/(precision+recall)

In [ ]:
print(F1_Score)